In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D,Dropout
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.xception import Xception, preprocess_input

In [2]:
!pip install py7zr

     |████████████████████████████████| 76 kB 562 kB/s            
     |████████████████████████████████| 2.0 MB 1.1 MB/s            
     |████████████████████████████████| 43 kB 1.1 MB/s             
     |████████████████████████████████| 131 kB 40.6 MB/s            
     |████████████████████████████████| 57 kB 3.9 MB/s             
     |████████████████████████████████| 2.4 MB 15.1 MB/s            
     |████████████████████████████████| 357 kB 33.2 MB/s            


In [3]:
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip',['.7z'],unpack_7zarchive) #Run for first time only

In [4]:
shutil.unpack_archive('../input/cifar-10/train.7z', '/kaggle/temp/')

In [5]:
train_labels = pd.read_csv("../input/cifar-10/trainLabels.csv", header="infer")

classes = train_labels['label'].unique()
print(classes)

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [6]:
if not os.path.exists("/kaggle/temp/valid"):
    os.mkdir("/kaggle/temp/valid")
    
parent_path_train = "/kaggle/temp/train"
parent_path_valid = "/kaggle/temp/valid"
parent_path_test = "/kaggle/temp/test"

for class1 in classes:
    path_train = os.path.join(parent_path_train,class1)
    if not os.path.exists(path_train):
        os.mkdir(path_train)
    path_valid = os.path.join(parent_path_valid,class1)
    if not os.path.exists(path_valid):
        os.mkdir(path_valid)
        
for (int_ind,row) in train_labels.iterrows():
    id = str(row["id"])+".png"
    source_path = os.path.join(parent_path_train,id)
    
    p=np.random.random()
    if p<=0.8:
        target_path = os.path.join(parent_path_train,row["label"],id)
        os.replace(source_path, target_path)
    else:
        target_path = os.path.join(parent_path_valid,row["label"],id)
        os.replace(source_path, target_path)

In [7]:
!ls /kaggle/temp/valid
!ls /kaggle/temp/train

airplane  automobile  bird  cat  deer  dog  frog  horse  ship  truck
airplane  automobile  bird  cat  deer  dog  frog  horse  ship  truck


In [8]:
xception_wo_top=Xception(include_top=False, input_shape=(291,291,3), weights='imagenet')
xception_wo_top.summary()

for layer in xception_wo_top.layers:
    layer.trainable=True

2022-04-07 18:23:58.702514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 18:23:58.843733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 18:23:58.845092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 18:23:58.849083: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

83697664/83683744 [==============================] - 3s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 291, 291, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 145, 145, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 145, 145, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 145, 145, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [9]:
classifier = Sequential()
classifier.add(xception_wo_top)
classifier.add(GlobalAveragePooling2D())
classifier.add(Dropout(0.3))
classifier.add(Dense(10,activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()

for i in classifier.layers:
    print(i.trainable)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 9, 9, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 20,881,970
Trainable params: 20,827,442
Non-trainable params: 54,528
_________________________________________________________________
True
True
True
True


In [10]:
reduceLROnPlateau=ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [11]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             preprocessing_function=preprocess_input,
                             rescale=None)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(directory='/kaggle/temp/train/', 
                                                    shuffle=True, target_size=(192,192),
                                                    batch_size=128)
valid_generator = valid_datagen.flow_from_directory(directory='/kaggle/temp/valid/',
                                                    shuffle=True, target_size=(192,192),
                                                    batch_size=128)

Found 40204 images belonging to 10 classes.
Found 9796 images belonging to 10 classes.


In [12]:
classifier.fit(train_generator,
               epochs=8, 
               validation_data=valid_generator,
               callbacks=[reduceLROnPlateau],
               steps_per_epoch=train_generator.n//train_generator.batch_size,
               validation_steps= valid_generator.n//valid_generator.batch_size,
               workers=8,
               use_multiprocessing=True)

2022-04-07 18:24:12.433276: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2022-04-07 18:24:30.603893: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-04-07 18:24:36.938027: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.68GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-07 18:24:37.421063: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.42GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-07 18:24:37.437242: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory

314/314 [==============================] - 488s 1s/step - loss: 0.3845 - accuracy: 0.8746 - val_loss: 0.4049 - val_accuracy: 0.8721
Epoch 2/8
314/314 [==============================] - 477s 1s/step - loss: 0.2007 - accuracy: 0.9326 - val_loss: 0.2700 - val_accuracy: 0.9153
Epoch 3/8
314/314 [==============================] - 484s 2s/step - loss: 0.1502 - accuracy: 0.9495 - val_loss: 0.2894 - val_accuracy: 0.9052
Epoch 4/8
314/314 [==============================] - 487s 2s/step - loss: 0.1275 - accuracy: 0.9581 - val_loss: 0.2698 - val_accuracy: 0.9186
Epoch 5/8
314/314 [==============================] - 485s 2s/step - loss: 0.1138 - accuracy: 0.9626 - val_loss: 0.2562 - val_accuracy: 0.9233
Epoch 6/8
314/314 [==============================] - 483s 2s/step - loss: 0.0908 - accuracy: 0.9697 - val_loss: 0.3566 - val_accuracy: 0.9010
Epoch 7/8
314/314 [==============================] - 488s 2s/step - loss: 0.0779 - accuracy: 0.9744 - val_loss: 0.2684 - val_accuracy: 0.9205
Epoch 8/8
314/31

In [13]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z','/kaggle/temp/test')

In [14]:
shutil.unregister_unpack_format('7zip')

In [15]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_gen = test_datagen.flow_from_directory(directory='/kaggle/temp/test',target_size=(192,192),batch_size=64,class_mode=None,shuffle=False)

Found 300000 images belonging to 1 classes.


In [16]:
test_gen.reset()
predictions_vecs = classifier.predict(test_gen)

predictions_final = np.argmax(predictions_vecs, axis=1)

In [17]:
print(type(train_generator.class_indices))
print(train_generator.class_indices)

classes = {value:key for (key,value) in train_generator.class_indices.items()}
print(classes)

predicted_classes=np.empty(shape=300000,dtype=np.dtype('U20'))

ind=0
for i in predictions_final.tolist():
    predicted_classes[ind]=classes[i]
    ind=ind+1
    
filenames_wo_ext = []
for fname in test_gen.filenames:
    filenames_wo_ext.append(int(fname.split(sep="/")[1].split(sep=".")[0])-1)

predicted_classes_final = np.empty(shape=300000,dtype=np.dtype('U20'))
predicted_classes_final[filenames_wo_ext]=predicted_classes

<class 'dict'>
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [18]:
sub = pd.read_csv('../input/cifar-10/sampleSubmission.csv',header='infer')
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      300000 non-null  int64 
 1   label   300000 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [19]:
sub['label'] = predicted_classes_final
sub.to_csv('19BCE226_submission.csv',index=False)